In [1]:
#### 패키지 임포트
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import os
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
pd.set_option('display.max_columns', None)

In [ ]:
#### 본문to형태소(morphs) 파일
df = pd.read_csv("../dataset/2022/본문_형태소분석/1월/나무문화_1월_형태소분석.csv", encoding='UTF-8')
print(df.shape, df.columns)
docs = df['칼럼명'].to_list()
print(len(docs))

In [ ]:






#### 각각의 본문이 str타입('나는 어제 밥을 먹었다')일 경우 list(['나', '는', '어제', '밥', '을' '먹다', '이다'])로 바꿔줌
docs2 = docs.copy()
for i in tqdm(range(len(docs))):
    docs2[i] = docs2[i].split()


#### 명사 빈도 상위 50개 단어 파일
topwords_df = pd.read_csv("파일경로", encoding='UTF-8')
print(topwords_df.shape, topwords_Df.columns)
topwords50 = topwords_df['칼럼명'].to_list()
print(len(topwords50))
print(topwords50)


%%time #소요시간 보려고 만든 거니까 없어도 됨.
#### 중복 단어 제거하여 리스트 생성
unique_lines = [list(set(line)) for line in tqdm(docs2)]


#### 중복 제거 잘 됐나 개수로 확인
print(len(docs[0]))
print(len(unique_lines[0]))
print(unique_lines[0][0:10])


#### 전체 단어쌍 빈도 dict 형식으로 생성
freq_count = {}   #동시출현 빈도가 저장될 dict
for words in tqdm(unique_lines):
    # 전체 단어에 대한 동시 출현 빈도
    for i, a in enumerate(words):
        for b in words[i+1:]:
            if a == b: continue
            elif a>b:
                freq_count[b, a] = freq_count.get((b, a),0) + 1
            else :
                freq_count[a, b] = freq_count.get((a, b),0) + 1



#### 딕셔너리를 데이터프레임에 담기
tt_freq_df=pd.DataFrame.from_dict(tt_freq_count, orient='index')


#### 딕트to데이터프레임 저장
freq_df.to_csv("파일저장경로/파일명.txt", encoding='UTF-8')



%%time #소요시간 보려고 만든 거니까 없어도 됨.
#### dict형식을 컬럼명 지정해서 데이터프레임으로 만들기
list1 = [(freq_df.index[i][0], freq_df.index[i][1], freq_df[0][i]) for i in tqdm(range(len(freq_df)))]
freq_df2 = pd.DataFrame(list1, columns=["term1","term2","freq"])
print(freq_df2.shape)
freq_df2.head()



#### 컬럼명 지정한 dict_to_데이터프레임 저장
freq_df2.to_csv("파일저장경로/파일명.txt", index=False, encoding='UTF-8')



%%time #소요시간 보려고 만든 거니까 없어도 됨.
#### 50위 이내에 속한 단어들로 이루어진 Dataframe을 추출
list2 =[(freq_df2.loc[i][0], freq_df2.loc[i][1], freq_df2.loc[i][2]) for i in tqdm(range(len(freq_df2))) if (freq_df2['term1'][i] in topwords50) &(freq_df2['term2'][i] in topwords50)]
freq_df3 = pd.DataFrame(list2, columns=["term1","term2","freq"])
freq_df3



#### 상위50위 단어들로 이루어진 단어쌍 데이터프레임 저장
freq_df3.to_csv("파일저장경로/파일명.txt", index=False, encoding='UTF-8')



#### 공출현빈도(1-mode matrix) 만들기
word_co_matx = np.zeros((50,50))
mat_idx = topwords50 # 단어동시출현행렬이니까 인덱스와 칼럼을 동일하게 지정
mat_col = topwords50
word_co_matx = pd.DataFrame(word_co_matx, index=mat_idx, columns=mat_col)
print(word_co_matx.shape)


#### 단어쌍의 첫 번째 단어가 인덱스와 같다 and 두 번째 단어가 칼럼과 같다 → 단어쌍빈도를 해당 셀 값에 삽입
for n in tqdm(range(len(freq_df3))):
    for idx in mat_idx:
        if idx == freq_df3['term1'][n]:
            for col in mat_col:
                if col == freq_df3['term2'][n]:
                    word_co_matx.loc[idx, col] = freq_df3['freq'][n]
                    word_co_matx.loc[col, idx] = freq_df3['freq'][n]



#### 1-mode 매트릭스 확인하기
tt_word_co_matx


#### 공출현빈도 데이터프레임 저장하기~~
#### ※ 인덱스가 저장돼야 하므로 index 파라미터 지정하지 말 것.
word_co_matx.to_csv("파일저장경로/파일명.txt", encoding='UTF-8')
word_co_matx.to_excel("파일저장경로/파일명.xlsx", encoding='UTF-8')
